In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns',100)
# Import modeling tools
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
# Set DataFrames as default output
from sklearn import set_config
import joblib
set_config(transform_output='pandas')

In [2]:
def regression_metrics(y_true, y_pred, label='', verbose=True,
                       output_dict=False):

  mae=mean_absolute_error(y_true, y_pred)
  mse= mean_squared_error(y_true, y_pred)
  rmse= mean_squared_error(y_true, y_pred, squared=False)
  r_squared= r2_score(y_true, y_pred)
  if verbose == True:

    header='-'*60
    print (header, f"Regression Metrics: {label}", header, sep='\n')
    print(f'-MAE = {mae:.3f}')
    print(f'-MSE={mse:,.3f}')
    print(f'-RMSE={rmse:,.3f}')
    print (f'-R^2={r_squared:,.3f}')
  if output_dict == True:
    metrics= {'Label':label, 'MAE':mae,
             'MSE':mse, 'RMSE':rmse, 'R^2':r_squared}
    return metrics

In [3]:
def evaluate_regression(reg, X_train, y_train,X_test, y_test, verbose= True,
                        output_frame=False):
  y_train_pred= reg.predict(X_train)

  results_train= regression_metrics(y_train, y_train_pred, verbose= verbose,
                                    output_dict= output_frame, label='Training Data')
  print()

  y_test_pred= reg.predict(X_test)

  results_test= regression_metrics(y_test, y_test_pred, verbose=verbose,
                                   output_dict=output_frame, label='Test Data')

  if output_frame:
    results_df= pd.DataFrame([results_train, results_test])

    results_df=results_df.set_index('Label')

    results_df.index.name= None

    return results_df.round(3)

In [4]:
import os, sys
%load_ext autoreload 
%autoreload 2
import custom_functions as fn

In [5]:
# Open the file structure dictionary

import json
with open('config/filepaths.json') as f:
    FPATHS = json.load(f)
FPATHS

{'data': {'raw': {'full': 'data/ames-housing-dojo-for-ml.csv',
   'eda': 'data/ames-housing-dojo-for-ml-eda.csv'},
  'ml': {'train': 'data/training-data.joblib',
   'test': 'data/testing-data.joblib'}},
 'models': {'linear_regression': 'models/linear_regression/linreg.joblib',
  'random_forest': 'models/random_forest/rf_reg.joblib'},
 'images': {'banner': 'images/app-banner.png'}}

In [6]:
# Define the filepath using the new dictionary structure
fpath = FPATHS['data']['raw']['full']
# Read in as normal
df_full = pd.read_csv(fpath)
df_full = df_full.set_index("PID")
df_full.head()

,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Utilities,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remodeled,Exter Qual,Exter Cond,Bsmt Unf Sqft,Total Bsmnt Sqft,Central Air,Living Area Sqft,Bedroom,Kitchen,Total Rooms,Garage Type,Garage Yr Blt,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Fence,SalePrice,Month,Year,Total Full Baths,Total Half Baths
PID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
907227090,RL,60.0,7200,Pave,NaN,AllPub,CollgCr,1Fam,1Story,5,8,1972,1972,TA,TA,427.0,864.0,Y,864.0,3,1,5,Detchd,1977.0,1.0,297.0,TA,TA,Y,MnPrv,119900.0,3,2006,1.0,0.0
527108010,RL,134.0,19378,Pave,NaN,AllPub,Gilbert,1Fam,2Story,7,5,2005,2006,Gd,TA,1335.0,1392.0,Y,2462.0,4,1,9,Attchd,2006.0,2.0,576.0,TA,TA,Y,NaN,320000.0,3,2006,3.0,1.0
534275170,RL,NaN,12772,Pave,NaN,AllPub,NAmes,1Fam,1Story,6,8,1960,1998,TA,Gd,460.0,958.0,Y,958.0,2,1,5,Attchd,1960.0,1.0,301.0,TA,TA,Y,NaN,151500.0,4,2007,1.0,0.0
528104050,RL,114.0,14803,Pave,NaN,AllPub,NridgHt,1Fam,1Story,10,5,2007,2008,Ex,TA,442.0,2078.0,Y,2084.0,2,1,7,Attchd,2007.0,3.0,1220.0,TA,TA,Y,NaN,385000.0,6,2008,3.0,0.0
533206070,FV,32.0,3784,Pave,Pave,AllPub,Somerst,TwnhsE,1Story,8,5,2006,2007,Gd,TA,1451.0,1511.0,Y,1565.0,2,1,5,Attchd,2006.0,2.0,476.0,TA,TA,Y,NaN,193800.0,2,2007,3.0,0.0


In [7]:
# Defining which columns to keep for model
target  = "SalePrice"
features_to_use =['Living Area Sqft', 'Lot Frontage', 'Bldg Type', 'Bedroom',
                      'Total Full Baths','MS Zoning','Street', 
                      'Alley','Utilities']

In [8]:
# Make the df_eda with the target and selected features
df_eda = df_full[[target, *features_to_use]].copy()
df_eda

,SalePrice,Living Area Sqft,Lot Frontage,Bldg Type,Bedroom,Total Full Baths,MS Zoning,Street,Alley,Utilities
PID,,,,,,,,,,
907227090,119900.0,864.0,60.0,1Fam,3,1.0,RL,Pave,NaN,AllPub
527108010,320000.0,2462.0,134.0,1Fam,4,3.0,RL,Pave,NaN,AllPub
534275170,151500.0,958.0,NaN,1Fam,2,1.0,RL,Pave,NaN,AllPub
528104050,385000.0,2084.0,114.0,1Fam,2,3.0,RL,Pave,NaN,AllPub
533206070,193800.0,1565.0,32.0,TwnhsE,2,3.0,FV,Pave,Pave,AllPub
...,...,...,...,...,...,...,...,...,...,...
903400030,109000.0,816.0,50.0,1Fam,2,1.0,RL,Pave,Pave,AllPub
533234020,223000.0,1789.0,79.0,1Fam,3,3.0,FV,Pave,NaN,AllPub
908188140,137500.0,1709.0,24.0,Twnhs,3,2.0,RM,Pave,NaN,AllPub


In [9]:
# Define file path with dictionary
eda_file_path = FPATHS['data']['raw']['eda']
eda_file_path

'data/ames-housing-dojo-for-ml-eda.csv'

In [10]:
# Now save the eda version of the dataset using the dictionary
df_eda.to_csv(eda_file_path)

In [11]:
# Separate features vs target & train/test split
target = "SalePrice"
X = df_eda.drop(columns = target)
y = df_eda[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
X_train

,Living Area Sqft,Lot Frontage,Bldg Type,Bedroom,Total Full Baths,MS Zoning,Street,Alley,Utilities
PID,,,,,,,,,
905475520,943.0,NaN,1Fam,3,1.0,RL,Pave,NaN,AllPub
909254010,1228.0,53.0,1Fam,3,1.0,RL,Pave,NaN,AllPub
531450090,1294.0,NaN,1Fam,3,3.0,RL,Pave,NaN,AllPub
903400040,1824.0,60.0,1Fam,3,1.0,RL,Pave,Pave,AllPub
527107130,1628.0,60.0,1Fam,3,2.0,RL,Pave,NaN,AllPub
...,...,...,...,...,...,...,...,...,...
527402220,1002.0,NaN,1Fam,3,2.0,RL,Pave,NaN,AllPub
528435030,1578.0,78.0,1Fam,3,2.0,RL,Pave,NaN,AllPub
528218090,1456.0,63.0,1Fam,3,2.0,RL,Pave,NaN,AllPub


In [12]:
# Defining the filepath for the training data using the dictionary
joblib_train_path = FPATHS['data']['ml']['train']
joblib_train_path

'data/training-data.joblib'

In [13]:
# Saving x-train and y-train a file using path from the dictionary
joblib.dump([X_train, y_train], joblib_train_path)

['data/training-data.joblib']

In [14]:
# Defining the filepath for the testing data using the dictionary
joblib_test_path = FPATHS['data']['ml']['test']
joblib_test_path

'data/testing-data.joblib'

In [15]:
# Saving x-test and y-test in a file using path from the dictionary
joblib.dump([X_test, y_test], joblib_test_path)

['data/testing-data.joblib']

In [16]:
# Make a preprocessing pipeline
# Separate lists of columns by dtype
num_cols = list(X_train.select_dtypes('number').columns)
ohe_cols = list(X_train.select_dtypes('object').columns)
# Categorical preprocessing pipeline (OHE)
impute_missing = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe_encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
ohe_pipe = make_pipeline(impute_missing, ohe_encoder)
# Numeric preprocessing pipeline
impute_nums = SimpleImputer(strategy='mean')
scaler = StandardScaler()
num_pipe = make_pipeline(impute_nums, scaler)
preprocessor = ColumnTransformer([('num', num_pipe, num_cols),
                                     ('cat',ohe_pipe, ohe_cols)],
                                    verbose_feature_names_out=False)
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer()),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 ['Living Area Sqft', 'Lot Frontage', 'Bedroom',
                                  'Total Full Baths']),
                                ('cat',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='MISSING',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False))]),
                                 ['Bldg Type', 'MS Zoning', 'Street', 'Alley',
                                  'Utilities'])],
                  verbose_feature_names_out=False)

In [17]:
# Linear regression model
# Define model pipeline for linear regression
lin_reg_pipe = Pipeline([
    ('preprocess', preprocessor),
    ('reg',LinearRegression())])
# Fit the model
lin_reg_pipe.fit(X_train, y_train)
# Make predictions and evaluate the model
results = evaluate_regression(lin_reg_pipe, X_train, y_train, X_test, y_test, output_frame=True)
results

NameError: name 'mean_absolute_error' is not defined

In [18]:
# Now we can save our model using the filepath from the dictionary
linreg_path = FPATHS['models']['linear_regression']
linreg_path

'models/linear_regression/linreg.joblib'

In [19]:
# Save linear regression model in a file using path from the dictionary
joblib.dump(lin_reg_pipe, linreg_path)

['models/linear_regression/linreg.joblib']

In [20]:
# Random Forest Pipeline
# Define model pipeline for random forest
rf_pipe = Pipeline([
    ('preprocess', preprocessor),
    ('reg',RandomForestRegressor(max_depth=8, 
                                 min_samples_leaf=2, 
                                 random_state=42))])
# Fit the model
rf_pipe.fit(X_train, y_train)
# Make predictions and evaluate the model
results =evaluate_regression(rf_pipe, X_train, y_train, X_test, y_test, output_frame=True)
results

NameError: name 'mean_absolute_error' is not defined

In [21]:
# Now we can save our model using the filepath from the dictionary
rf_path = FPATHS['models']['random_forest']
rf_path

'models/random_forest/rf_reg.joblib'

In [22]:
# Save random forest model in a file using path from the dictionary
joblib.dump(lin_reg_pipe, rf_path)

['models/random_forest/rf_reg.joblib']